### Convert timezone using python

Objective:
* Find timezone based on longtidue and latitude
* Change UTC time to local time

Packages:
* [timezonefinder](https://pypi.org/project/timezonefinder/)
* [pandas](https://pandas.pydata.org/)

In [2]:
# Import packages
from timezonefinder import TimezoneFinder
import pandas as pd

`timezonefinder` is used to identify local time zone based on longitude and latitude. To save computing time at the cost of memory consumption and initialisation time pass `in_memory=True`. This causes all binary files to be read into memory. 

In [15]:
# Find timezone based on longitude and latitude
tf = TimezoneFinder(in_memory=True)
longitude = -88
latitude = 36
local_time_zone = tf.timezone_at(lng=longitude, lat=latitude)
local_time_zone

'America/Chicago'

Here we will create a DatetimeIndex (April 06 - 07, 2019 with frequency of 4 hours) using pandas [date_range](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.date_range.html) function.

In [23]:
# Create naive timestamps using pandas
test_naive = pd.date_range('2019-04-06', '2019-04-07', freq='4H')
test_naive

DatetimeIndex(['2019-04-06 00:00:00', '2019-04-06 04:00:00',
               '2019-04-06 08:00:00', '2019-04-06 12:00:00',
               '2019-04-06 16:00:00', '2019-04-06 20:00:00',
               '2019-04-07 00:00:00'],
              dtype='datetime64[ns]', freq='4H')

As we can see from the above output, the timestamps are in naive format, i.e. without timezone information. A typical problem we usually encounter is that the above timestamps is under UTC for some observations or model results, and we'd like to convert it to local time zone in order to calcualte some local information (e.g. daily precipitation). To do that, we first need to add time zone information for the naive timestamps, e.g., we can set them to UTC (assume they are actually in UTC) using the [tz_localize](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.tz_localize.html) function from pandas.

In [24]:
# Set time to be UTC
test_UTC = test_naive.tz_localize('UTC')
test_UTC

DatetimeIndex(['2019-04-06 00:00:00+00:00', '2019-04-06 04:00:00+00:00',
               '2019-04-06 08:00:00+00:00', '2019-04-06 12:00:00+00:00',
               '2019-04-06 16:00:00+00:00', '2019-04-06 20:00:00+00:00',
               '2019-04-07 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='4H')

The output now has additional time zone information (e.g. `+00:00`). Then we can convert it to local time genrated by longitude and latidue at the beginning using the [tz_convert](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.tz_convert.html) function.

In [25]:
# Convert UTC to local time
test_local = test_UTC.tz_convert(local_time_zone)
test_local

DatetimeIndex(['2019-04-05 19:00:00-05:00', '2019-04-05 23:00:00-05:00',
               '2019-04-06 03:00:00-05:00', '2019-04-06 07:00:00-05:00',
               '2019-04-06 11:00:00-05:00', '2019-04-06 15:00:00-05:00',
               '2019-04-06 19:00:00-05:00'],
              dtype='datetime64[ns, America/Chicago]', freq='4H')

Apparently, the time zone now is changed to local time automatically, i.e. `-05:00` instead of `+00:00`. Sometimes, we also need to convert the timestamps with timezonen information back to naive timestamps but still in local time zone. We can use [tz_localize](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.tz_localize.html) function again, but this time the argument is `None` which means naive timestamps.

In [28]:
# Convert back to naive timestamps, but in local time zone.
test_local_naive = test_local.tz_localize(None)
test_local_naive

DatetimeIndex(['2019-04-05 19:00:00', '2019-04-05 23:00:00',
               '2019-04-06 03:00:00', '2019-04-06 07:00:00',
               '2019-04-06 11:00:00', '2019-04-06 15:00:00',
               '2019-04-06 19:00:00'],
              dtype='datetime64[ns]', freq='4H')

Now, we have convert the UTC time to local time and the format is the same, i.e. naive timestamps. Functions we used in this conversion are

* [tf.timezone_at](https://pypi.org/project/timezonefinder/): get time zone based on longitude and latidue
* [tz_localize](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.tz_localize.html): convert naive timestamps to timestamps with time zone information, or the reverse process
* [tz_convert](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.tz_convert.html): convert timestamps between different time zones